In [141]:
import pandas as pd

churn_df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

churn_df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [142]:
churn_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [143]:
churn_df['TotalCharges'] = pd.to_numeric(churn_df['TotalCharges'], errors='coerce')

In [144]:
churn_df.isnull().sum()

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [147]:
ortalama_deger = churn_df['TotalCharges'].mean()

churn_df['TotalCharges'] = churn_df['TotalCharges'].fillna(ortalama_deger)

In [148]:
churn_df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [150]:
object_sutunlar = churn_df.select_dtypes(include = ['object']).columns

for sutun in object_sutunlar:
    print(f"--- Sütun Adı: {sutun} ---")
    print(churn_df[sutun].value_counts())
    print("\n" + "="*30 + "\n")

--- Sütun Adı: customerID ---
customerID
7590-VHVEG    1
3791-LGQCY    1
6008-NAIXK    1
5956-YHHRX    1
5365-LLFYV    1
             ..
9796-MVYXX    1
2637-FKFSY    1
1552-AAGRX    1
4304-TSPVK    1
3186-AJIEK    1
Name: count, Length: 7043, dtype: int64


--- Sütun Adı: gender ---
gender
Male      3555
Female    3488
Name: count, dtype: int64


--- Sütun Adı: Partner ---
Partner
No     3641
Yes    3402
Name: count, dtype: int64


--- Sütun Adı: Dependents ---
Dependents
No     4933
Yes    2110
Name: count, dtype: int64


--- Sütun Adı: PhoneService ---
PhoneService
Yes    6361
No      682
Name: count, dtype: int64


--- Sütun Adı: MultipleLines ---
MultipleLines
No                  3390
Yes                 2971
No phone service     682
Name: count, dtype: int64


--- Sütun Adı: InternetService ---
InternetService
Fiber optic    3096
DSL            2421
No             1526
Name: count, dtype: int64


--- Sütun Adı: OnlineSecurity ---
OnlineSecurity
No                     3498
Yes    

In [151]:
churn_df['Churn'] = churn_df['Churn'].map({'No':0, 'Yes':1})

In [152]:
churn_df['Churn'].value_counts()

Churn
0    5174
1    1869
Name: count, dtype: int64

In [153]:
binary_cols = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']

for sutun in binary_cols:
    churn_df[sutun] = churn_df[sutun].map({'No':0, 'Yes':1})

In [154]:
churn_df['gender'] = churn_df['gender'].map({'Female': 0, 'Male': 1})

In [155]:
churn_df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,0,0,1,0,1,0,No phone service,DSL,No,...,No,No,No,No,Month-to-month,1,Electronic check,29.85,29.85,0
1,5575-GNVDE,1,0,0,0,34,1,No,DSL,Yes,...,Yes,No,No,No,One year,0,Mailed check,56.95,1889.50,0
2,3668-QPYBK,1,0,0,0,2,1,No,DSL,Yes,...,No,No,No,No,Month-to-month,1,Mailed check,53.85,108.15,1
3,7795-CFOCW,1,0,0,0,45,0,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,0,Bank transfer (automatic),42.30,1840.75,0
4,9237-HQITU,0,0,0,0,2,1,No,Fiber optic,No,...,No,No,No,No,Month-to-month,1,Electronic check,70.70,151.65,1


In [158]:
kategorik_sutunlar = churn_df.select_dtypes(include=['object']).columns.drop(['customerID'])

churn_df_final = pd.get_dummies(churn_df, columns=kategorik_sutunlar, drop_first=True)

churn_df_final = churn_df_final.drop('customerID', axis=1)

In [159]:
churn_df_final.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,...,TechSupport_Yes,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,0,1,0,1,0,1,29.85,29.85,0,...,False,False,False,False,False,False,False,False,True,False
1,1,0,0,0,34,1,0,56.95,1889.50,0,...,False,False,False,False,False,True,False,False,False,True
2,1,0,0,0,2,1,1,53.85,108.15,1,...,False,False,False,False,False,False,False,False,False,True
3,1,0,0,0,45,0,0,42.30,1840.75,0,...,True,False,False,False,False,True,False,False,False,False
4,0,0,0,0,2,1,1,70.70,151.65,1,...,False,False,False,False,False,False,False,False,True,False


In [160]:
churn_df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 31 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   gender                                 7043 non-null   int64  
 1   SeniorCitizen                          7043 non-null   int64  
 2   Partner                                7043 non-null   int64  
 3   Dependents                             7043 non-null   int64  
 4   tenure                                 7043 non-null   int64  
 5   PhoneService                           7043 non-null   int64  
 6   PaperlessBilling                       7043 non-null   int64  
 7   MonthlyCharges                         7043 non-null   float64
 8   TotalCharges                           7043 non-null   float64
 9   Churn                                  7043 non-null   int64  
 10  MultipleLines_No phone service         7043 non-null   bool   
 11  Mult

In [161]:
X = churn_df_final.drop('Churn', axis=1)

y = churn_df_final['Churn']

In [162]:
X.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,MultipleLines_No phone service,...,TechSupport_Yes,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,0,1,0,1,0,1,29.85,29.85,True,...,False,False,False,False,False,False,False,False,True,False
1,1,0,0,0,34,1,0,56.95,1889.50,False,...,False,False,False,False,False,True,False,False,False,True
2,1,0,0,0,2,1,1,53.85,108.15,False,...,False,False,False,False,False,False,False,False,False,True
3,1,0,0,0,45,0,0,42.30,1840.75,True,...,True,False,False,False,False,True,False,False,False,False
4,0,0,0,0,2,1,1,70.70,151.65,False,...,False,False,False,False,False,False,False,False,True,False


In [163]:
y.head()

0    0
1    0
2    1
3    0
4    1
Name: Churn, dtype: int64

In [164]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.25,random_state=42)

In [165]:
print(f"X_train (Eğitim Soruları) şekli: {X_train.shape}")
print(f"X_test (Test Soruları) şekli: {X_test.shape}")
print(f"\n y_train (Eğitim Cevapları) şekli: {y_train.shape}")
print(f"y_test (Test Cevapları) şekli: {y_test.shape}")

X_train (Eğitim Soruları) şekli: (5282, 30)
X_test (Test Soruları) şekli: (1761, 30)

 y_train (Eğitim Cevapları) şekli: (5282,)
y_test (Test Cevapları) şekli: (1761,)


In [167]:
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression(max_iter=1000)

log_model.fit(X_train, y_train)

print("Model başarıyla eğitildi!")

Model başarıyla eğitildi!


/opt/miniconda3/envs/ai_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [173]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)

In [169]:
log_model.fit(X_train_scaled, y_train)

print("Model, ölçeklendirilmiş veri ile başarıyla yeniden eğitildi!")

Model, ölçeklendirilmiş veri ile başarıyla yeniden eğitildi!


In [171]:
y_pred = log_model.predict(X_test_scaled)

print(f"Modelin Tahminleri : {y_pred[:20]}")
print(f"Gerçek Değerler : {y_test.to_numpy()[:20]}")

Modelin Tahminleri : [1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0]
Gerçek Değerler : [1 0 0 1 0 1 0 0 1 1 0 0 0 1 0 1 1 1 0 1]


In [174]:
dogruluk = log_model.score(X_test_scaled, y_test)

print(f"\n Modelin Test Setindeki Doğruluk Oranı: {dogruluk}")
print(f" Modelin Başarısı: %{dogruluk * 100:.2f}")


 Modelin Test Setindeki Doğruluk Oranı: 0.8131743327654741
 Modelin Başarısı: %81.32


In [175]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test,y_pred)

print("\n Karmaşıklık Matrisi:\n",cm)


 Karmaşıklık Matrisi:
 [[1154  128]
 [ 201  278]]
